# Selección de características en modelos predictivos

Este proyecto es para IA de Moisés Calzado Cobo y Antonio Germán Márquez Trujillo

Estos archivos contienen los algoritmos implementados, pero no los importamos ya que en esta libreta los implementamos explicando su funcionamiento


In [1]:
import pandas as pandas
import time
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp
from sklearn.model_selection import cross_val_score
from sklearn import tree
from funcy import join
from sklearn import preprocessing
pandas.set_option('max_colwidth', 800)

In [19]:
#datos = pandas.read_csv("datos/titanic.csv")
datos = pandas.read_csv("datos/BreastCancerDataset.csv")
datos.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [1]:
def mostrar_datos(datos):
    datos_tabla = [['Mejor Solución', 'Tamaño', 'Rendimiento']]
    claves = datos.keys()
    
    for clave in claves:
        length = len(clave.split(", "))
        datos_tabla = datos_tabla +[[clave, length, datos[clave]]]

    return pandas.DataFrame(datos_tabla)

In [35]:
def evaluar_soluciones(datos, solucion_actual, nueva_variable, objetivo, n_exp, cv):
    variables = solucion_actual[:]
    variables.append(nueva_variable)
    data_frame = pandas.DataFrame(data=datos)
    X = data_frame[variables]
    y = data_frame[objetivo]
    clf = tree.DecisionTreeClassifier()
    
    scores = cross_val_score(clf, X, y, cv=cv, scoring="balanced_accuracy")
    
    for i in range(n_exp-1):
        new_scores = cross_val_score(clf, X, y, cv=cv, scoring="balanced_accuracy") 
        scores = scores + new_scores
    
    scores = scores/n_exp
    
    diccionario_resultado = {}
    diccionario_resultado[nueva_variable] = np.mean(scores)
    return diccionario_resultado

In [21]:
def evaluar_soluciones_eliminando(datos, solucion_actual, variable_a_eliminar, objetivo, n_exp, cv):
    variables = solucion_actual[:]
    variables.remove(variable_a_eliminar)
    if len(variables) < 1:
        return
    data_frame = pandas.DataFrame(data=datos)
    X = data_frame[variables]
    y = data_frame[objetivo]
    clf = tree.DecisionTreeClassifier()
    
    scores = cross_val_score(clf, X, y, cv=cv, scoring="balanced_accuracy")
    
    for i in range(n_exp-1):
        new_scores = cross_val_score(clf, X, y, cv=cv, scoring="balanced_accuracy") 
        scores = scores + new_scores
    
    scores = scores/n_exp
    
    diccionario_resultado = {}
    diccionario_resultado[variable_a_eliminar] = np.mean(scores)
    return diccionario_resultado

Algoritmo SFS paralelizado:

In [6]:
def SFS(datos, respuesta, d = 0):
    start = time.time()

    diccionario_resultado = {}
    columnas = list(datos.columns)
    columnas.remove(respuesta)
    solucion_actual = []

    k = 0
    d = d if d else len(columnas)
    
    while(k<d):
        pool = mp.Pool(mp.cpu_count())
        new_resultados = pool.starmap(evaluar_soluciones, [(datos, solucion_actual, nuevaVariable, respuesta, 15, 10) for nuevaVariable in columnas])
        pool.close()
        resultado = join(new_resultados)
        
        variable_escogida = max(resultado, key=resultado.get)
        variables_nuevas = variable_escogida.split(", ")
        solucion_actual.append(variables_nuevas[len(variables_nuevas)-1])
        columnas.remove(variables_nuevas[len(variables_nuevas)-1])

        k = k+1
        
        diccionario_resultado[variable_escogida] = resultado[variable_escogida]
    
    done = time.time()
    elapsed = done - start
    print("Tiempo empleado: ", elapsed)
    return diccionario_resultado

In [ ]:
resultado_SFS = SFS(datos, "diagnosis", 3)

Algoritmo SFFS paralelizado:

In [38]:
def SFFS(datos, respuesta):
    start = time.time()
    diccionario_resultado = {}
    soluciones_actual = []
    añadidos = []
    eliminados = []
    columnas = list(datos.columns)
    k = 0
    
    #Compruebo que la variable a predecir no esté en mi conjunto de variables a evaluar
    if respuesta in columnas:
        columnas.remove(respuesta)
    
    while(k<10):
        resultado = []
        score_resultado = 0
        score_resultado_eliminado = 0
        resultado_eliminado = []
        eliminado = ''
        
        #Actualizo el listado de columnas que tengo que evaluar
        columnas_a_evaluar = [x for x in columnas if (x not in añadidos and x not in soluciones_actual)]
                
        if columnas_a_evaluar == []:
            break
        
        #Calculamos el nuevo resultado óptimo
        pool = mp.Pool(mp.cpu_count())
        new_resultados = pool.starmap(evaluar_soluciones, [(datos, soluciones_actual, nuevaVariable, respuesta, 15, 10) for nuevaVariable in columnas_a_evaluar])
        pool.close()
        #Buscamos la variable más óptima
        resultado = join(new_resultados)
        variable_escogida = max(resultado, key=resultado.get)
        score_resultado = resultado[variable_escogida]
        #Añadimos la variable
        soluciones_actual.append(variable_escogida)
        añadidos.append(variable_escogida)
        
        # Pasamos a eliminar variables para comprobar si tengo mejores resultados
        if len(soluciones_actual) > 1:
            variables_a_eliminar = [x for x in soluciones_actual if x not in eliminados]

            pool = mp.Pool(2)
            new_resultados = pool.starmap(evaluar_soluciones_eliminando, [(datos, soluciones_actual, variable_a_eliminar, respuesta, 15, 10) for variable_a_eliminar in variables_a_eliminar])
            pool.close()
            resultado = join(new_resultados)
            variable_a_eliminar = max(resultado, key=resultado.get)
            
            score_resultado_eliminado = resultado[variable_a_eliminar]
            
            if score_resultado < score_resultado_eliminado:
                soluciones_actual.remove(variable_a_eliminar)
                eliminados.append(variable_a_eliminar)
                score_resultado = score_resultado_eliminado
                k = 0
                        
        if len(columnas) == len(datos.columns)-1:
            k = k+1
        
        if len(añadidos)<len(columnas):
            clave = ', '.join(soluciones_actual)
            diccionario_resultado[clave] = score_resultado

    done = time.time()
    elapsed = done - start
    print("Tiempo empleado: ", elapsed)
    return diccionario_resultado 

In [39]:
resultado_SFFS = SFFS(datos, "diagnosis")

0.7487987012987014
0.8330050505050505
0.5851262626262625
0.80507215007215
0.7144372294372294
0.8054004329004328
0.5305916305916307
0.5623340548340549
0.590306637806638
0.7916919191919192
0.8210497835497836
0.5415728715728716
0.716381673881674
0.6115007215007215
0.770450937950938
0.6164574314574314
0.5071536796536796
0.6060497835497837
0.4890909090909091
0.8448484848484847
0.8719444444444445
0.549498556998557
0.49429653679653673
0.8702886002886003
0.5871212121212122
0.8797691197691198
0.7960425685425686
0.692994227994228
0.6377813852813853
0.5687049062049062

0.8889136604136605
0.8911457431457432
0.8747881192881192
0.8626344396344396
0.8908994708994709
0.8517554112554112
0.8933631553631555
0.887979797979798
0.8645755170755169
0.8706839826839827
0.8763821548821549
0.8609752284752284
0.8870752765752767
0.8617607022607021
0.9010646945646945
0.8568210678210679
0.8716878306878308
0.8633340548340549
0.8793501683501683
0.9159752284752285
0.8656339586339585
0.9093987493987493
0.8930567580567581

0.9405110630110629
0.9364973544973545
0.9494723424723424
0.935092833092833

0.9364973544973545
0.9494723424723424
0.935092833092833
0.943736652236652
0.9516137566137568
0.9405110630110629
0.9524379509379507

0.943736652236652
0.9364973544973545
0.9494723424723424
0.935092833092833
0.9405110630110629
0.9516137566137568

0.943736652236652
0.9494723424723424
0.9405110630110629
0.9364973544973545
0.935092833092833

0.9364973544973545
0.943736652236652
0.935092833092833
0.9405110630110629

0.9364973544973545
0.9405110630110629
0.935092833092833

0.9364973544973545
0.935092833092833

0.935092833092833

Tiempo empleado:  109.16689705848694


In [41]:
mostrar_datos(resultado_SFFS)

,0,1,2
0,Mejor Solución,Tamaño,Rendimiento
1,worst concave points,1,0.879769
2,"worst concave points, worst radius",2,0.915975
3,"worst concave points, worst radius, mean texture",3,0.940305
4,"worst concave points, worst radius, mean texture, worst smoothness",4,0.946353
5,"worst concave points, worst radius, mean texture, worst smoothness, concavity error",5,0.948617
6,"worst radius, mean texture, worst smoothness, concavity error, worst concavity",5,0.951864
7,"worst radius, mean texture, worst smoothness, concavity error, worst concavity, area error",6,0.954557
8,"worst radius, mean texture, worst smoothness, concavity error, worst concavity, area error, mean perimeter",7,0.955903
9,"worst radius, mean texture, worst smoothness, concavity error, worst concavity, area error, mean perimeter, compactness error",8,0.956815
